# Тестирование модуля формирования портфеля

Этот ноутбук демонстрирует полный цикл работы с модулем 	vr_service.pipeline.portfolio — от загрузки данных по инструментам до расчёта распределения капитала и экспорта результатов.

## 1. Подготовка окружения

Импортируем необходимые библиотеки и подключим модуль портфеля. Добавляем каталог src в sys.path, чтобы можно было использовать код проекта без установки пакета.

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from tvr_service.pipeline import (
    allocations_to_frame,
    build_portfolio,
    filter_by_suffix,
    filter_by_whitelist,
    load_securities,
    load_whitelist,
)

import pandas as pd

## 2. Настройка параметров теста

Задаём базовые параметры: объём капитала, необходимость скачивания свежих данных с MOEX, путь к локальному CSV (если он есть) и фильтрацию по суффиксу тикера. Все параметры можно менять и перезапускать ячейки.

In [2]:
CAPITAL = 3_00_000  # общий капитал, руб.
PREFER_REMOTE = True   # при True сначала пробуем скачать данные с MOEX
SEC_DATA_FILE = None   # можно указать путь к локальному sec_tvr.csv
SUFFIX_FILTER = 'Z5'   # например, 'H5', 'M5' и т.п. для фьючерсов


## 3. Загрузка справочника инструментов

Попробуем получить таблицу инструментов. Функция load_securities автоматически рассчитает поле 

ull_price и добавит тикеры TI* (клонов TB*). В процессе будет использован список из интернета или локального файла.

In [3]:
securities = load_securities(sec_data_file=SEC_DATA_FILE, prefer_remote=PREFER_REMOTE)
securities.head()

,SECID,MINSTEP,STEPPRICE,PREVSETTLEPRICE,INITIALMARGIN,BUYSELLFEE,SCALPERFEE,SHORTNAME,full_price,CODE,PRICE,SELLDEPO,base_code
0,AEH6,0.001,1.0,24.550,3859.08,1.13,0.57,AED-3.26,24550.0,AEH6,24.550,3859.08,AED
1,AEM6,0.001,1.0,25.120,4040.33,1.16,0.58,AED-6.26,25120.0,AEM6,25.120,4040.33,AED
2,AEZ5,0.001,1.0,23.478,3759.46,1.08,0.54,AED-12.25,23478.0,AEZ5,23.478,3759.46,AED
3,AFH6,1.000,1.0,6020.000,1611.87,1.20,0.60,AFLT-3.26,6020.0,AFH6,6020.000,1611.87,AFLT
4,AFZ5,1.000,1.0,5793.000,1523.15,1.15,0.58,AFLT-12.25,5793.0,AFZ5,5793.000,1523.15,AFLT


In [4]:
securities.shape

(400, 13)

### Новые поля справочника

Справочник теперь содержит дополнительные столбцы:

* `SHORTNAME` — полное имя инструмента, как его отдаёт ISS.
* `CODE` — копия `SECID` (для совместимости со старым кодом).
* `PRICE` и `SELLDEPO` — котировка и гарантийное обеспечение для расчётов.
* `base_code` — базовое имя (до дефиса), используется при фильтрации whitelist.

## 4. Проверка whitelist

Сначала ограничиваем список инструментов по выбранному `SUFFIX_FILTER`, затем применяем whitelist. Так можно сузить вселенную до нужной серии и далее оставить только интересующие базовые активы.

In [5]:
try:
    whitelist = load_whitelist()
    print(f'Инструментов в whitelist: {len(whitelist)}')
except FileNotFoundError:
    whitelist = None
    print('Файл whitelist не найден. Будем использовать полную вселенную инструментов.')

base_selection = filter_by_suffix(securities, SUFFIX_FILTER)
print(f'Инструментов после фильтра по суффиксу: {len(base_selection)}')

if whitelist and not base_selection.empty:
    filtered_by_whitelist = filter_by_whitelist(base_selection, whitelist)
    print(f'Инструментов после применения whitelist: {len(filtered_by_whitelist)}')
elif whitelist:
    filtered_by_whitelist = pd.DataFrame(columns=base_selection.columns if not base_selection.empty else securities.columns)
    print('После фильтра по суффиксу подходящих инструментов нет.')
else:
    filtered_by_whitelist = base_selection

if not filtered_by_whitelist.empty:
    print('Примеры совпадений:')
    display(filtered_by_whitelist[['SECID', 'SHORTNAME', 'base_code']].head())
else:
    print('Совпадений не найдено.')

Инструментов в whitelist: 44
Инструментов после фильтра по суффиксу: 131
Инструментов после применения whitelist: 45
Примеры совпадений:


,SECID,SHORTNAME,base_code
0,AFZ5,AFLT-12.25,AFLT
1,AKZ5,AFKS-12.25,AFKS
2,ALZ5,ALRS-12.25,ALRS
3,BNZ5,BANE-12.25,BANE
4,CMZ5,CBOM-12.25,CBOM


In [6]:
filtered_by_whitelist

,SECID,MINSTEP,STEPPRICE,PREVSETTLEPRICE,INITIALMARGIN,BUYSELLFEE,SCALPERFEE,SHORTNAME,full_price,CODE,PRICE,SELLDEPO,base_code
0,AFZ5,1.0,1.0,5793.0,1523.15,1.15,0.58,AFLT-12.25,5793.0,AFZ5,5793.0,1523.15,AFLT
1,AKZ5,1.0,1.0,14687.0,3857.79,2.91,1.46,AFKS-12.25,14687.0,AKZ5,14687.0,3857.79,AFKS
2,ALZ5,1.0,1.0,4627.0,1214.01,0.92,0.46,ALRS-12.25,4627.0,ALZ5,4627.0,1214.01,ALRS
3,BNZ5,1.0,1.0,1600.0,544.83,0.31,0.16,BANE-12.25,1600.0,BNZ5,1600.0,544.83,BANE
4,CMZ5,1.0,1.0,7539.0,2576.72,1.49,0.75,CBOM-12.25,7539.0,CMZ5,7539.0,2576.72,CBOM
5,FLZ5,1.0,1.0,8367.0,1541.90,1.65,0.83,FLOT-12.25,8367.0,FLZ5,8367.0,1541.90,FLOT
6,FSZ5,1.0,1.0,6850.0,1463.69,1.36,0.68,FEES-12.25,6850.0,FSZ5,6850.0,1463.69,FEES
7,GKZ5,1.0,1.0,1277.0,235.32,0.26,0.13,GMKN-12.25,1277.0,GKZ5,1277.0,235.32,GMKN
8,GZZ5,1.0,1.0,12441.0,2287.73,2.47,1.24,GAZR-12.25,12441.0,GZZ5,12441.0,2287.73,GAZR
9,HDZ5,1.0,1.0,3543.0,1209.21,0.70,0.35,HEAD-12.25,3543.0,HDZ5,3543.0,1209.21,HEAD


In [5]:
try:
    whitelist = load_whitelist()
    print(f'Инструментов в whitelist: {len(whitelist)}')
except FileNotFoundError:
    whitelist = None
    print('Файл whitelist не найден. Будем использовать полную вселенную инструментов.')

if whitelist:
    filtered_by_whitelist = filter_by_whitelist(securities, whitelist)
    print(f'Инструментов после применения whitelist: {len(filtered_by_whitelist)}')
    print('Примеры совпадений:')
    print(filtered_by_whitelist[['SECID', 'SHORTNAME', 'base_code']].head())
else:
    filtered_by_whitelist = securities


Инструментов в whitelist: 30
Инструментов после применения whitelist: 51
Примеры совпадений:
  SECID   SHORTNAME base_code
0  AFH6   AFLT-3.26      AFLT
1  AFZ5  AFLT-12.25      AFLT
2  AKH6   AFKS-3.26      AFKS
3  AKZ5  AFKS-12.25      AFKS
4  BNH6   BANE-3.26      BANE


## 5. Построение портфеля

Выполним расчёт: функция uild_portfolio вернёт список PortfolioEntry, содержащий распределённый капитал и оценку количества лотов. Затем конвертируем результат в DataFrame для удобного анализа.

In [6]:
portfolio_entries = build_portfolio(
    capital=CAPITAL,
    suffix=SUFFIX_FILTER,
    whitelist=whitelist,
    sec_data_file=SEC_DATA_FILE,
    prefer_remote=PREFER_REMOTE,
)

portfolio_df = allocations_to_frame(portfolio_entries)
portfolio_df.head()

,SECID,allocation,estimated_lots,full_price,used_capital,unused_capital
0,AFZ5,11111.111111,1,6083.0,6083.0,5028.111111
1,AKZ5,11111.111111,0,15027.0,0.0,11111.111111
2,BNZ5,11111.111111,6,1687.0,10122.0,989.111111
3,FLZ5,11111.111111,1,8604.0,8604.0,2507.111111
4,FSZ5,11111.111111,1,6795.0,6795.0,4316.111111


In [7]:
portfolio_df

,SECID,allocation,estimated_lots,full_price,used_capital,unused_capital
0,AFZ5,11111.111111,1,6083.0,6083.0,5028.111111
1,AKZ5,11111.111111,0,15027.0,0.0,11111.111111
2,BNZ5,11111.111111,6,1687.0,10122.0,989.111111
3,FLZ5,11111.111111,1,8604.0,8604.0,2507.111111
4,FSZ5,11111.111111,1,6795.0,6795.0,4316.111111
5,GKZ5,11111.111111,8,1261.0,10088.0,1023.111111
6,IRZ5,11111.111111,0,32128.0,0.0,11111.111111
7,LKZ5,11111.111111,0,63284.0,0.0,11111.111111
8,MCZ5,11111.111111,1,7565.0,7565.0,3546.111111
9,MEZ5,11111.111111,0,17660.0,0.0,11111.111111


## 6. Диагностика распределения капитала

Проверим, сколько средств было фактически распределено по инструментам, и какой объём остался неиспользованным из-за округления до целых лотов.

In [8]:
allocated_capital = portfolio_df['used_capital'].sum()
unused_capital = portfolio_df['unused_capital'].sum()
print(f'Фактически распределено: {allocated_capital:,.2f} руб.')
print(f'Неиспользованный остаток: {unused_capital:,.2f} руб.')
print(f'Доля остатка: {unused_capital / CAPITAL:.2%}')


Фактически распределено: 159,497.00 руб.
Неиспользованный остаток: 140,503.00 руб.
Доля остатка: 46.83%


## 7. Топ инструментов по задействованному капиталу

Отсортируем таблицу по полю used_capital, чтобы понять, какие инструменты потребляют наибольшую долю капитала.

In [ ]:
portfolio_df.sort_values('used_capital', ascending=False).head(10)

## 8. Сохранение результатов

По желанию можно сохранить расчёт в CSV. По умолчанию файлы складываются в каталог docs.

In [ ]:
output_path = PROJECT_ROOT / 'docs' / 'portfolio_result.csv'
portfolio_df.to_csv(output_path, index=False)
output_path

## 9. Следующие шаги

* Изменяйте параметры и фильтры, чтобы тестировать разные сценарии.
* Добавьте собственные таблицы параметров, когда они будут готовы, и объедините их с расчётами.
* Используйте ноутбук как основу для презентации результатов или автоматизации отчётов.

In [3]:
whitelist = load_whitelist()

In [4]:
whitelist

['AFKS',
 'AFLT',
 'BANE',
 'FEES',
 'FLOT',
 'GMKN',
 'IRAO',
 'LKOH',
 'MAGN',
 'MGNT',
 'MOEX',
 'MTLR',
 'MVID',
 'NLMK',
 'NOTK',
 'PHOR',
 'PIKK',
 'PLZL',
 'RTKM',
 'RUAL',
 'SGZH',
 'SIBN',
 'SMLT',
 'SNGP',
 'SOFL',
 'TATN',
 'TCSI',
 'TRNF',
 'VKCO',
 'WUSH']